In [1]:
import pandas as pd
import pickle, numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
from utils import verifyDir

### Reading images

In [15]:
import glob

In [11]:
DATA_PATH = "data/PlacePulse_2/images/"
MAIN_OUT_PATH = "outputs/"

OUT_DETECT_PATH = f"{MAIN_OUT_PATH}segmentations/"

OUT_IMAGE_PATH = f"{MAIN_OUT_PATH}segmented_images/"

In [12]:
verifyDir(MAIN_OUT_PATH)
verifyDir(OUT_DETECT_PATH)
verifyDir(OUT_IMAGE_PATH)

In [26]:
files_downloaded = [file for file in glob.glob(f'{DATA_PATH}/*/*', recursive = True)]

In [30]:
files_downloaded, len(files_downloaded)

(['data/PlacePulse_2/images/Houston/29.740200_-95.284784_513cc058fdc9f0358700137e_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.658276_-95.372664_513cc646fdc9f03587001930_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.660180_-95.370341_513d35bffdc9f03587002b1e_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.660599_-95.370350_513cc072fdc9f0358700139b_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.661336_-95.385899_513cc0eafdc9f03587001413_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.662314_-95.385725_513cc3a8fdc9f035870016be_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.662849_-95.361930_513cbf18fdc9f0358700123a_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.662913_-95.370398_513cc42cfdc9f03587001738_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.663270_-95.356580_513cba32fdc9f03587000dae_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.664332_-95.357103_513d365ffdc9f03587002b92_Houston.JPG',
  'data/PlacePulse_2/images/Houston/29.6

### Extracting features

In [7]:
from utils.sceneparsers.DeepLabV3 import DeepLabV3_Xception, ProcessSegmentation, ade20k_map_color_label

2022-12-12 13:06:26.030261: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-12-12 13:06:26.030274: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-12 13:06:26.416572: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 13:06:26.437206: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 3187200000 Hz
2022-12-12 13:06:26.438623: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3430ad0 initialized for platform Host (this does not guarantee 

In [31]:
already_tested=True

In [32]:
names, colors = ade20k_map_color_label()

In [33]:
colors[list(names.keys())[list(names.values()).index("tree")]]

array([  4., 200.,   3.])

In [38]:
if not already_tested:
    segment_model = DeepLabV3_Xception()

2022-12-12 13:32:40.836305: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 13:32:40.836426: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2022-12-12 13:32:40.836530: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/felipe/anaconda3/envs/segmentator/lib/python3.7/site-packages/cv2/../../lib64:
2022-12-12 13:32:40.836567: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcublas.so.10'; dlerror: libcub

In [43]:
if not already_tested:
    for img_name in files_downloaded:
        city_name = img_name.split("/")[3]
        img_out = img_name.split("/")[-1].replace(".JPG", "")
        out_csv = f"{OUT_DETECT_PATH}/{city_name}/"
        out_image = f"{OUT_IMAGE_PATH}/{city_name}/"
        
        verifyDir(out_csv)
        verifyDir(out_image)

        _, _, masks = ProcessSegmentation(segment_model, img_name, output_image_name=f"{out_image}/{img_out}.png", overlay=True)

        masks_df = pd.DataFrame(data=masks)
        masks_df.to_csv(f"{out_csv}/{img_out}.csv", index=False)

### Organizing files

In [47]:
len(glob.glob(f'{OUT_DETECT_PATH}/*/*.csv')), len(glob.glob(f'{OUT_IMAGE_PATH}/*/*.png'))

(110988, 110988)

### Summarizing results

In [48]:
already_summized = False

In [49]:
if not already_summized:
    img_id = {"image_id":[]}

    for k,v in names.items():
        img_id[v] = []

In [53]:
if not already_summized:
    for img_name in files_downloaded:
        city_name = img_name.split("/")[3]
        img_out = img_name.split("/")[-1].replace(".JPG", "")
        out_csv = f"{OUT_DETECT_PATH}/{city_name}/"
        masks_df = pd.read_csv(f"{out_csv}/{img_out}.csv", low_memory=False)
        #masks_df = masks_df[masks_df["ratios"]>94.0].copy()

        img_id["image_id"].append(img_out)

        classes_list = list(masks_df["names"].values)
        name_list = list(masks_df["names"].values)
        ratio_list = list(masks_df["ratios"].values)

        for name in list(names.values()):
            if name in name_list:
                idx_name = name_list.index(name)
                img_id[name_list[idx_name]].append(ratio_list[idx_name])
            else:
                img_id[name].append(0)

In [55]:
if not already_summized:
    segment_city_df = pd.DataFrame(data=img_id)

In [56]:
if not already_summized:
    print("Saving ...", f"{MAIN_OUT_PATH}objects_deeplab_xception.csv")
    segment_city_df.to_csv(f"{MAIN_OUT_PATH}objects_deeplab_xception.csv", sep=";", index=False)

Saving ... outputs/objects_deeplab_xception.csv


In [57]:
if already_summized:
    segment_city_df = pd.read_csv(f"{MAIN_OUT_PATH}objects_deeplab_xception.csv", sep=";")